In [ ]:
from math import inf
import sys
import os
import time
import random
HUMAN = 1
COMP = -1
VOID = 0
LEGEND = dict(zip((VOID, HUMAN, COMP), (" ", "X", "O")))

def evaluate(board):
    """
    Perform heuristic evaluation from board.
    Heuristic - allow the computer to discover the solution
    of some problems by itself.
    """
    if wins(board, COMP):
        return COMP
    if wins(board, HUMAN):
        return HUMAN
    return VOID

def empty_cells(board):
    """Extract the remainder of board"""
    cells = []  # it contains all empty cells
    # Use enumerate for easy indexing
    for i, row in enumerate(board):
        for j, col in enumerate(row):
            if board[i][j] == VOID:
                cells.append((i, j))
    return cells

# Row calc example to split wins code
def win_row(board, player, nb_win_case):
    for row in board:
        isRow = 0
        for col in row:
            if col == player:
                isRow += 1
                if isRow == nb_win_case:
                    return True
            # Reset isRow on Void cases
            elif isRow:
                isRow = 0
    return False

# Col calc example to split wins code
def win_col(board, player, nb_win_case):
    isCol = [0 for _ in range(0, len(board))]
    for row in board:
        for y, col in enumerate(row):
            if col == player:
                isCol[y] += 1
                if isCol[y] == nb_win_case:
                    return True
            # Reset isCol[col] on Void cases
            elif isCol[y]:
                isCol[y] = 0
    return False

# Diag calc example to split wins code
def win_diag(board, player, nb_win_case):
    for x, row in enumerate(board):
        if x + nb_win_case > len(board):
            break
        for y, col in enumerate(row):
            # Cherche dans y meme si y + nb_win_case > len(board) car il cherche dans diagonal gauche et droite en un seul for
            if col == player:
                isDiagL = 0
                isDiagR = 0
                for i in range(0, nb_win_case):
                    if y + i < len(board) and board[x+i][y+i] == player:
                        isDiagL += 1
                    if y >= i and board[x+i][y-i] == player:
                        isDiagR += 1
                if isDiagL == nb_win_case or isDiagR == nb_win_case:
                    return True

def wins(board, player, nb_win_case: int = 4):
    if nb_win_case > len(board):
        nb_win_case = len(board)
    # version splitté en plusieurs fonction mais moins optimisé
    # return win_row(board, player, nb_win_case) \
    #     or win_col(board, player, nb_win_case) \
    #     or win_diag(board, player, nb_win_case)
    isCol = []
    for x, row in enumerate(board):
        isRow = 0
        for y, col in enumerate(row):
            # Init cols on first row
            if x == 0:
                isCol.append(0)
            if col == player:
                # Check row
                isRow += 1
                if isRow == nb_win_case:
                    return True
                # Check cols
                isCol[y] += 1
                if isCol[y] == nb_win_case:
                    return True
                # Check digonals
                if x + nb_win_case <= len(board):
                    isDiagL = 0
                    isDiagR = 0
                    for i in range(0, nb_win_case):
                        if y + i < len(board) and board[x+i][y+i] == player:
                            isDiagL += 1
                        if y >= i and board[x+i][y-i] == player:
                            isDiagR += 1
                    if isDiagL == nb_win_case or isDiagR == nb_win_case:
                        return True
            else:
                isRow = 0
                isCol[y] = 0
    return False

def game_over(board):
    """Check game over condition"""
    return wins(board, HUMAN) or wins(board, COMP)

def clean():
    """Clear system terminal"""
    os_name = sys.platform.lower()
    if os_name.startswith('win'):
        os.system('cls')
    else:
        os.system('clear')

 # Min-max

def minimax(board, depth, player):
    # inf/-inf are the initial score for the players
    best = [None, None, inf if player == COMP else -inf]
    if depth == 0 or game_over(board):
        return [None, None, evaluate(board)]
        # return [None, None, evaluate(board) * ( 1 / depth if depth else 1)]
    for cell in empty_cells(board):
        # Fill the empty cells with the player symbols
        x, y = cell[0], cell[1]
        board[x][y] = player
        if evaluate(board) == COMP:
            best = [ x, y, COMP]
            board[x][y] = 0
            break
        score = minimax(board, depth - 1, -player)
        board[x][y] = 0
        score[0], score[1] = x, y
        if player == COMP:
            if score[2] < best[2]:
                best = score
        elif score[2] > best[2]:
            best = score
    return best

# Min-max avec Alpha Bêta

def minimaxWithAB(board, depth, player, alpha = -inf, beta = inf):
    # inf/-inf are the initial score for the players
    best = [None, None, inf if player == COMP else -inf]
    if depth == 0 or game_over(board):
        return [None, None, evaluate(board)]
        # return [None, None, evaluate(board) * ( 1 / depth if depth else 1)]
    for cell in empty_cells(board):
        # Fill the empty cells with the player symbols
        x, y = cell[0], cell[1]
        board[x][y] = player
        if evaluate(board) == COMP:
            best = [ x, y, COMP]
            board[x][y] = 0
            break
        score = minimaxWithAB(board, depth - 1, -player, alpha, beta)
        board[x][y] = 0
        score[0], score[1] = x, y
        if player == COMP:
            if score[2] < best[2]:
                best = score
            if best[2] <= alpha:
                return best
            if best[2] < beta:
                beta = best[2]
        else:
            if score[2] > best[2]:
                best = score
            if best[2] >= beta:
                return best
            if best[2] > alpha:
                alpha = best[2]
    return best

# board : [
#     [0,0,0], 0 * 3 + 1 = 1
#     [0,0,0], 1 * 3 + 1 = 4
#     [0,0,0], 2 * 3 + 1 = 7
# ]

def nb_to_coord(move: int, board_length: int):
    for i in range(0, board_length):
        for j in range(0, board_length):
            if move == ((i * board_length) + j + 1):
                return i, j
    return None

def human_turn(board):
    nb_cases = len(board)
    remain = empty_cells(board)
    isTurn = True
    print("Your Turn")
    while isTurn:
        try:
            move = int(
                input(f"Enter your move (1-{nb_cases * nb_cases}) :")
            )
            coord = nb_to_coord(move, nb_cases)
            # When the player move is valid
            if coord in remain:
                x, y = coord
                board[x][y] = 1
                isTurn = False
            else:
                print("This case is full, try again.")
        # When the player mistype
        except ValueError:
            print(
                f"Wrong input, please enter (1-{nb_cases * nb_cases})"
            )
    # While-else loop, this code below will run after successful loop.
    else:
        # Clean the terminal, and show the current board
        # clean()
        print(render(board))


# argument 1 : board ; argument 2 : type d'ia ; argument 3 : choix du mode du jeu, par défaut 1 ; argument 4 : Croix ou Rond sur la grille, par défaut 1 : argument 5 : valeur de simulation, par défaut 1
def ai_turn(board,ai_choice,game_mode_choice=1,starting_ia=1,simulation_value=1):

    # IA Min-max
    if (ai_choice == 1):
      if (simulation_value == 1):
        if (game_mode_choice == 1):
          print("AI Turn : \n")
        else:
          print("Minmax AI Turn : \n")
      depth = len(empty_cells(board))  # The remaining of empty cells
      # the optimal move for computer
      row, col, score = minimax(board, depth, COMP)
      
      # Si IA VS IA, on vérifie si elle joue en premier ou deuxième
      if (game_mode_choice == 2):
        board[row][col] = starting_ia
      else:
        board[row][col] = COMP
      if (simulation_value == 1):  
        print(render(board))  # Show result board

    # IA Naïve (Reprend la fonction human_turn)
    if (ai_choice == 2):
      if (simulation_value == 1):
        if (game_mode_choice == 1):
          print("AI Turn : \n")
        else:
          print("Naive AI Turn : \n")
      nb_cases = len(board)
      remain = empty_cells(board)
      # La case sur laquelle va jouer l'IA naïve est aléatoire, comprise entre 1 et le nombre de cases du board
      move = random.randint(1, nb_cases * nb_cases)
      coord = nb_to_coord(move, nb_cases)
      # Étant donné que la case est choisie de façon aléatoire, en fin de partie les chances de tomber sur une case déjà remplie sont grandes, donc on relance jusqu'à tomber sur une case vide
      if coord in remain:
                  x, y = coord
                  if (game_mode_choice == 1):
                    board[x][y] = -1
                  else:
                    board[x][y] = starting_ia
      else:
                  while coord not in remain:
                    move = random.randint(0, nb_cases * nb_cases)
                    coord = nb_to_coord(move, nb_cases)
                  if coord in remain:
                    x, y = coord
                    if (game_mode_choice == 1):
                      board[x][y] = -1
                    else:
                      board[x][y] = starting_ia
      if (simulation_value == 1):                
        print(render(board))           

    # IA alphabeta
    if (ai_choice == 3):
        if (simulation_value == 1):
          if (game_mode_choice == 1):
            print("AI Turn : \n")
          else:
            print("Alphabeta AI Turn : \n")
        depth = len(empty_cells(board))  # The remaining of empty cells
        # the optimal move for computer
        row, col, score = minimaxWithAB(board, depth, COMP)
        
        # Si IA VS IA, on vérifie si elle joue en premier ou deuxième
        if (game_mode_choice == 2):
          board[row][col] = starting_ia
        else:
          board[row][col] = COMP
        if (simulation_value == 1):
          print(render(board))  # Show result board

def render(board):
    """Render the board board to stdout"""
    pretty_board = [[LEGEND[col] for col in row] for row in board]
    return ("{}\n" * len(pretty_board)).format(*pretty_board)

def make_board(nb: int = 3):
    return [[0 for j in range(0, nb)] for i in range(0, nb)]
    # return [[0]*nb]*nb // Ne marche, chaque ligne obtient la même id()

# Lance le jeu
def main():
    print(
        "\nWelcome to Tic Tac Toe.\n"
        "Play against our AI or observe 2 AI play against each other.\n"
    )
    clean()

    # Choix du nombre de cases (pas optimisé pour plus de 3 cases)
    nb_cases_ok = False
    while not nb_cases_ok:
        try:
            nb_cases = int(input("Choose the number of cases : "))
            nb_cases_ok = True
        except ValueError:
            print("You should write a number")

    # Choix du mode de jeu : Joueur vs IA, ou IA vs IA
    game_mode = False
    while not game_mode:
        try:
            game_mode_choice = int(input("Choose the game mode : 1 for Player VS AI, 2 for AI VS AI : "))
            game_mode = True
        except ValueError:
            print("You should write a number")

    # Choix de l'IA pour le joueur vs IA : minmax ou random
    if (game_mode_choice == 2):
      ai_choice_ok = True
    else:
      ai_choice_ok = False
    while not ai_choice_ok:
        try:
            ai_choice = int(input("Choose the type of AI : 1 for minmax, 2 for naive, 3 for alphabeta : "))
            ai_choice_ok = True
        except ValueError:
            print("You should write a number")

    # Décider qui commence pour IA VS IA
    if (game_mode_choice != 2):
      first_move = True
    else:
      first_move = False
    while not first_move:
      try:
        first_move_ai = int(input("Choose who plays first : 1 for minmax AI, 2 for naive AI, 3 for alphabeta AI : "))
        first_move = True
      except ValueError:
        print("You should write a number")

    # Décider qui joue en deuxième pour IA VS IA
    if (game_mode_choice != 2):
      second_move = True
    else:
      second_move = False
    while not second_move:
      try:
        second_move_ai = int(input("Choose who plays second : 1 for minmax AI, 2 for naive AI, 3 for alphabeta AI : "))
        second_move = True
      except ValueError:
        print("You should write a number")

    # Décider si on simule plusieurs parties ou pas
    if (game_mode_choice != 2):
      simulation = True
      simulation_value = 1
    else:
      simulation = False
    while not simulation:
      try:
        simulation_value = int(input("Do you want to start a simulation ? 1 for no, higher value for yes : "))
        simulation = True
      except ValueError:
        print("You should write a number")

    print("Game is ready !\n")
    
    # Variables de simulation
    i = 1
    first_ai_wins = 0
    second_ai_wins = 0
    draws = 0
    while i <= simulation_value:
      board = make_board(nb_cases)
      # On affiche le board uniquement pendant une partie unique
      if (simulation_value == 1):
        print(render(board), end="\n")

      while not wins(board, COMP) and len(empty_cells(board)) > 0:
        ############################ Tour de celui qui commence #######################
        # Si mode de jeu 1, c'est le joueur qui joue
          if (game_mode_choice == 1):
            human_turn(board)
          # Si mode de jeu 2 (IA minmax contre IA Alphabeta)
          else:
            ai_turn(board,first_move_ai,game_mode_choice,1,simulation_value)

          ##################### Tour de celui qui joue en deuxième ####################
          if len(empty_cells(board)) == 0 or wins(board, HUMAN):
              break
          start_time = time.time()
          if (game_mode_choice == 1):
            ai_turn(board,ai_choice)
          else:
            ai_turn(board,second_move_ai,game_mode_choice,-1,simulation_value)
          #print("--- %s seconds ---" % (time.time() - start_time))

      # Vérifie qui a gagné la partie
      if wins(board, COMP):
        if (game_mode_choice == 1):
          print("AI wins !")
        else:
          if (second_move_ai == 1):
            print("Minmax AI wins !")
          elif (second_move_ai == 2):
            print("Naive AI wins !")
          elif (second_move_ai == 3):
            print("Alphabeta AI wins !")
          # Pour le résultat si simulation
          second_ai_wins += 1
      elif wins(board, HUMAN):
        if (game_mode_choice == 1):
          print("You win !")
        else:
          if (first_move_ai == 1):
            print("Minmax AI wins !")
          elif (first_move_ai == 2):
            print("Naive AI wins !")
          elif (first_move_ai == 3):
            print("Alphabeta AI wins !")
          # Pour le résultat si simulation
          first_ai_wins += 1
      else:
          print("It's a Draw. No one wins")
          # Pour le résultat si simulation
          draws += 1
      i += 1

    # Résultat de la simulation
    if (simulation_value > 1):
      if (first_move_ai == 1):
        first_ai = "Minmax AI"
      elif (first_move_ai == 2):
        first_ai = "Naive AI"
      elif (first_move_ai == 3):
        first_ai = "Alphabeta AI"
      if (second_move_ai == 1):
        second_ai = "Minmax AI"
      elif (second_move_ai == 2):
        second_ai = "Naive AI"
      elif (second_move_ai == 3):
        second_ai = "Alphabeta AI"
      print(f"Simulation result : {first_ai} : {first_ai_wins} win(s), {second_ai} : {second_ai_wins} win(s), {draws} draw(s).")
    
    # Possibilité de relancer une partie
    replay = input("Do you want to play again ? (y/n) ")
    if (replay == "y" or replay == "yes"):
      clean()
      main()
    else:
      clean()

if __name__ == '__main__':
    main()